In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# load data from JSON file
with open('intents.json') as file:
    data = json.load(file)

# extract text and intent from data
texts = []
intents = []
for intent in data['intents']:
    for text in intent['text']:
        texts.append(text)
        intents.append(intent['intent'])

# tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
encoded_texts = tokenizer.texts_to_sequences(texts)

# save tokenizer
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# pad sequences to have equal length
max_len = max([len(x) for x in encoded_texts])
padded_texts = pad_sequences(encoded_texts, maxlen=max_len, padding='post')

# create label encoder object
le = LabelEncoder()

# fit and transform the intents to integer labels
encoded_intents = le.fit_transform(intents)

# get the number of unique labels
num_intents = len(le.classes_)

# apply one-hot encoding to the integer labels
encoded_intents = tf.one_hot(encoded_intents, depth=num_intents)

# define model architecture
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_len)(input_layer)
lstm_layer = LSTM(128)(embedding_layer)
output_layer = Dense(num_intents, activation='softmax')(lstm_layer)
model = Model(inputs=input_layer, outputs=output_layer)

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train model
model.fit(padded_texts, encoded_intents, epochs=50, batch_size=16)

# save model
model.save('chatbot_model.h5')

Epoch 1/50
26/26 [==============================] - 4s 17ms/step - loss: 3.6102 - accuracy: 0.0642
Epoch 2/50
26/26 [==============================] - 0s 14ms/step - loss: 3.4861 - accuracy: 0.0691
Epoch 3/50
26/26 [==============================] - 0s 13ms/step - loss: 3.2263 - accuracy: 0.1111
Epoch 4/50
26/26 [==============================] - 0s 14ms/step - loss: 2.8975 - accuracy: 0.1802
Epoch 5/50
26/26 [==============================] - 0s 14ms/step - loss: 2.6473 - accuracy: 0.2074
Epoch 6/50
26/26 [==============================] - 0s 13ms/step - loss: 2.2949 - accuracy: 0.3086
Epoch 7/50
26/26 [==============================] - 0s 14ms/step - loss: 1.9325 - accuracy: 0.4198
Epoch 8/50
26/26 [==============================] - 0s 14ms/step - loss: 1.6031 - accuracy: 0.5407
Epoch 9/50
26/26 [==============================] - 0s 13ms/step - loss: 1.2678 - accuracy: 0.6543
Epoch 10/50
26/26 [==============================] - 0s 15ms/step - loss: 0.9755 - accuracy: 0.7358
Epoch 11/

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# load data from JSON file
with open('intents_final.json') as file:
    data = json.load(file)

# extract text and intent from data
texts = []
intents = []
for intent in data['intents']:
    for text in intent['text']:
        texts.append(text)
        intents.append(intent['intent'])

# tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# load saved model
model = load_model('chatbot_final.h5')

# define maximum sequence length
max_len = model.input_shape[1]

# create label encoder object
le = LabelEncoder()
le.fit(intents)

# create inverse mapping of label encoder for intent prediction
intent_mapping = {i: label for i, label in enumerate(le.classes_)}

# start chatbot interaction
print('Welcome to the chatbot! Type "quit" to exit.')
while True:
    # get user input
    user_input = input('You: ').lower().strip()
    
    # check if user wants to quit
    if user_input == 'quit':
        break
    
    # encode user input text
    encoded_input = tokenizer.texts_to_sequences([user_input])
    padded_input = pad_sequences(encoded_input, maxlen=max_len, padding='post')
    
    # predict intent
    intent_prob = model.predict(padded_input)[0]
    intent_idx = np.argmax(intent_prob)
    intent_label = le.inverse_transform([intent_idx])[0]
    
    # retrieve response
    for intent in data['intents']:
        if intent['intent'] == intent_label:
            response = np.random.choice(intent['responses'])
            print('Chatbot:', response)
            break

Welcome to the chatbot! Type "quit" to exit.
You: Hello
1/1 [==============================] - 1s 547ms/step
Chatbot: Good to see you again!
You: Where is HOD?
1/1 [==============================] - 0s 19ms/step
Chatbot: All engineering departments have only one hod XYZ who available on (Place name)
You: Where is cafateria?
1/1 [==============================] - 0s 19ms/step
Chatbot: Good to see you again!
You: Where is Cafetaria?
1/1 [==============================] - 0s 18ms/step
Chatbot: Our university has canteen with variety of food available
You: how many departments are there in JUIT
1/1 [==============================] - 0s 18ms/step
Chatbot: <a target="_blank" href="ADD YOU GOOGLE MAP LINK HERE"> here</a>


KeyboardInterrupt: ignored